In [16]:
import os
import pandas as pd

os.chdir(r"C:\Users\GilseungAhn\Google 드라이브\작업\온라인 강의 머신러닝 성능 향상을 위한 데이터 탐색과 전처리 심화\데이터\실습 데이터 - #3 ~ #5")

In [17]:
df = pd.read_csv("australian.csv")

In [18]:
# 특징과 라벨 분리
X = df.drop('Class', axis = 1)
Y = df['Class']

In [19]:
# 학습 데이터와 평가 데이터 분리
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [20]:
Train_X.shape # 샘플 대비 특징이 약간 더 많은 정도

(517, 14)

In [21]:
# 특징 선택 전 성능 확인
from sklearn.neighbors import KNeighborsClassifier as KNN
model = KNN().fit(Train_X, Train_Y)
pred_Y = model.predict(Test_X)

from sklearn.metrics import f1_score
print(f1_score(Test_Y, pred_Y))

0.5714285714285713


In [22]:
# 유니크한 값의 개수를 바탕으로 연속형과 이진형 변수 구분
continuous_cols = [col for col in Train_X.columns if len(Train_X[col].unique()) > 3]
binary_cols = [col for col in Train_X.columns if len(Train_X[col].unique()) <= 3]

In [23]:
# 연속형 변수에 대해서는 f_classif을, 이진형 변수에 대해서는 chi2를 적용
from sklearn.feature_selection import *

# f_regression(X, Y) => (statistics, p-value)
continous_cols_pvals = f_classif(Train_X[continuous_cols], Train_Y)[1]
binary_cols_pvals = chi2(Train_X[binary_cols], Train_Y)[1]

In [24]:
# 각각을 Series로 변환 (value: pvalue, index: colum name)
cont_pvals = pd.Series(continous_cols_pvals, index = continuous_cols)
binary_pvals = pd.Series(binary_cols_pvals, index = binary_cols)

In [25]:
# cont_pvals과 binary_pvals을 합친 뒤, 오름차순으로 정렬 (앞에 나오는 특징부터 좋은 특징)
pvals = pd.concat([cont_pvals, binary_pvals])
pvals.sort_values(ascending = True, inplace = True)

In [26]:
# 특징 선택 
k = 10

s_Train_X = Train_X[pvals.iloc[:k].index]
s_Test_X = Test_X[pvals.iloc[:k].index]

In [27]:
# 특징 선택 후 성능 확인
from sklearn.neighbors import KNeighborsClassifier as KNN
model = KNN().fit(s_Train_X, Train_Y)
pred_Y = model.predict(s_Test_X)

from sklearn.metrics import f1_score
print(f1_score(Test_Y, pred_Y))

0.6225165562913907
